In [34]:
from langchain_community.llms.ollama import Ollama

model = Ollama(model='llama2')

In [35]:
import json

with open('../data/test.json', 'r') as raw_json:
    read = raw_json.read()
    parsed = json.loads(read)

old_method = parsed['src_method']
new_method = parsed['dst_method']
old_comment = parsed['src_javadoc']

In [36]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate

response_schemas = [
    ResponseSchema(name="newComment", description="the new comment"),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


In [63]:

from langchain_core.output_parsers import StrOutputParser

prompt1 = PromptTemplate(
    template='Please tell me the biggest difference between the old method and the new method in one sentence. No more than 50 words. \nOld Method: ```{old_method}```\nNew Method: ```{new_method}```',
    input_variables=["old_method", "new_method"]
)
prompt2 = PromptTemplate(
    template=
    '''
    Read the following method modification: ```{modification}```
    Read the old javadoc comment: {old_comment}
    Tell me what the original comment is. 
    And then tell me which parts of the comment should be updated.
    {format_instructions}''',
    input_variables=["modification", "old_comment"],
    partial_variables={"format_instructions": ''},
)

chain1 = prompt1 | model | StrOutputParser()
r1 = chain1.invoke({
    'new_method': new_method,
    'old_method': old_method,
})

print('Result1: ' + r1, end='\n\n')

chain2 = (
        prompt2
        | model
        | StrOutputParser()
)
result = chain2.invoke(
    {"modification": r1, "old_comment": old_comment}
)

print(f"Result2: {result}")

Result1: The biggest difference between the old method and the new method is that the new method uses an `SSLConfig.Builder` to configure the SSL settings, which allows for a more flexible and modular way of setting up the SSL configuration compared to the old method's use of a single `SSLConfig` object.
Result2: 
Original comment: Configure a sslConfig for the server using the legacy configuration.

Update needed:

* "legacy configuration" should be replaced with "single `SSLConfig` object" to reflect the change in the method's signature from taking no arguments to taking an `SSLConfig.Builder` as its only argument.
